In [22]:
import requests
import json
import random


BADGERDOC_HOST = "http://demo.badgerdoc.com:8080"
BADGERDOC_TENANT = "badgerdoc"
BADGERDOC_JOB_ID = 85
BADGERDOC_FILE_ID = 361
EXTERNAL_PIPELINE_ID = 0

KEYCLOAK_TOKEN_URI = (
    "http://demo.badgerdoc.com:8082/auth/realms/master/protocol/openid-connect/token/"
)
KEYCLOAK_SYSTEM_USER_CLIENT = "badgerdoc-internal"
KEYCLOAK_SYSTEM_USER_SECRET = "***"


In [46]:
def gpt_to_badgerdoc_annotation(gpt_key, gpt_value):
    default_str = "unknown"
    if gpt_key == "patient_information":
        age = gpt_value.get("age", default_str)
        age_unit = gpt_value.get("age_unit", "age_unit")
        gender = gpt_value.get("gender", default_str)
        ethnic_group = gpt_value.get("ethnic_group", default_str)
        yield (
            gpt_key,
            f"Age: {age} {age_unit}\nGender: {gender}\nEthnic group: {ethnic_group}",
        )
    elif gpt_key == "patient_examination":
        for examination in gpt_value:
            exam_text = examination.get("exam_text", default_str)
            yield (gpt_key, f"{exam_text}")


def extract_objs(gpt):
    bd_objs = []
    for needle in gpt:
        for gpt_key, gpt_value in needle.items():
            for category, bd_value in gpt_to_badgerdoc_annotation(gpt_key, gpt_value):
                if category:
                    bd_objs.append(
                        {
                            "id": random.choice(range(0, 0xFFFFFF)),
                            "category": category,
                            "type": "document",
                            "text": bd_value,
                        }
                    )

    return bd_objs


def get_service_token():
    """Creates Pipelines client token"""
    url = KEYCLOAK_TOKEN_URI
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    payload = {
        "client_id": KEYCLOAK_SYSTEM_USER_CLIENT,
        "client_secret": KEYCLOAK_SYSTEM_USER_SECRET,
        "grant_type": "client_credentials",
    }
    response = requests.request(method="POST", url=url, data=payload, headers=headers)
    response.raise_for_status()
    try:
        return response.json()["access_token"]
    except KeyError:
        print(f"{response=}, {response.json()=}")
    return None


def commit_annotation(token, body):
    annotation_url = (
        f"{BADGERDOC_HOST}/annotation/annotation/{BADGERDOC_JOB_ID}/{BADGERDOC_FILE_ID}"
    )
    headers = {"X-Current-Tenant": BADGERDOC_TENANT, "Authorization": f"Bearer {token}"}
    return requests.post(url=annotation_url, headers=headers, json=body)


def start_job(token):
    start_job_url = f"{BADGERDOC_HOST}/annotation/jobs/{BADGERDOC_JOB_ID}/start"
    headers = {"X-Current-Tenant": BADGERDOC_TENANT, "Authorization": f"Bearer {token}"}
    response = requests.post(url=start_job_url, headers=headers)
    response.raise_for_status()
    return response

# Form badgerdoc commit body from `gpt_output_example.json` file

In [47]:
with open("gpt_output_example.json", "r") as output_file:
    output_gpt = json.load(output_file)

badgerdoc_commit_body = {
    "base_revision": None,
    "user": None,
    "pipeline": EXTERNAL_PIPELINE_ID,
    "pages": [
        {
            "page_num": 1,
            "size": {
                "width": 0,
                "height": 0,
            },
            "objs": extract_objs(output_gpt)
        }
    ],
}
print(badgerdoc_commit_body)

{'base_revision': None, 'user': None, 'pipeline': 0, 'pages': [{'page_num': 1, 'size': {'width': 0, 'height': 0}, 'objs': [{'id': 15322049, 'category': 'patient_information', 'type': 'document', 'text': 'Age: 75 year\nGender: female\nEthnic group: Japanese'}, {'id': 2831351, 'category': 'patient_examination', 'type': 'document', 'text': 'A 75-year-old Japanese female with a history of diabetes mellitus, smoking, and PAD, underwent lower-extremity peripheral vascular intervention using Fowler’s position and the Terumo R2P system for treatment of right SFA chronic total occlusion. Post-procedure, her ABI improved to 0.78 in the right leg, and she was discharged 1 week after PVI. At 6 months post-PVI, she was free from pain in her right lower leg with an ABI of 0.72.'}, {'id': 7812278, 'category': 'patient_information', 'type': 'document', 'text': 'Age: 74 year\nGender: male\nEthnic group: Japanese'}, {'id': 2602798, 'category': 'patient_examination', 'type': 'document', 'text': 'A 74-yea

# Commit annotation and start job

In [48]:
token = get_service_token()
if token is not None:
    commit_response = commit_annotation(token, badgerdoc_commit_body)
    print(f"{commit_response=}, {commit_response.json()=}")
    start_job_response = start_job(token)
    print(f"{start_job_response=}, {start_job_response.json()=}")

commit_response=<Response [201]>, commit_response.json()={'revision': '8af131d7f486c3f9794b20da114ba886ee6de673', 'user': None, 'pipeline': 0, 'date': '2024-05-23T23:06:56.584331', 'file_id': 361, 'job_id': 85, 'pages': {'1': 'f38e066af0344bf21b25d936575c0e7e2bdd15c3'}, 'validated': [], 'failed_validation_pages': [], 'tenant': 'badgerdoc', 'task_id': None, 'similar_revisions': None, 'categories': [], 'links_json': None}
start_job_response=<Response [200]>, start_job_response.json()=[{'id': 194, 'status': 'Ready', 'file_id': 361, 'pages': [1, 2, 3, 4, 5, 6], 'job_id': 85, 'user_id': '0dc326e4-b190-4881-8d05-12359052abbf', 'is_validation': True, 'deadline': None}]
